In [ ]:
from ipywidgets import *

In [ ]:
import traitlets
from six import text_type

In [ ]:
class EmptyCell(Button):
    def __init__(self, width='20px', height='20px'):
        super(EmptyCell, self).__init__(layout=Layout(
                width=width,
                height=height,
                visibility='hidden')
        )

class TextCell(Text):
    displaytype = text_type
    def __init__(self, *arg, **kwargs):
        super(TextCell, self).__init__(*arg, **kwargs)
        self.continuous_update = False

class AddableTextCell(TextCell):
    def __init__(self, **kwargs):
        super(AddableTextCell, self).__init__(**kwargs)
        self.add_class('addable_cell')
    

In [ ]:
class Property:
    def __init__(self, basename, cond=lambda x: x):
        self.basename = basename
        self.cond = cond

        self.css_name = '%s_%X' % (basename, id(cond))

    def has_property(self, name, value):
        return name == self.basename and self.cond(value)

    def get_css_name(self):
        return self.css_name

    def eval(self, value=True):
        return (self.basename, value)

In [ ]:
class ObjectWithProperties:
    def __init__(self, values):
        self.value = dict(values)


In [ ]:
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter
from sage_combinat_widgets.grid_view_editor import GridViewEditor

In [ ]:
class ObjectGridViewAdapter(GridViewAdapter):
    objclass = ObjectWithProperties
    celltype = int
    cellzero = False
    
    @staticmethod
    def compute_cells(obj):
        cells = { (i,j):obj.value[i,j] for i,j in obj.value }
        return cells
    
    @staticmethod
    def get_properties(obj):
        values = obj.value

        props = {}

        for i,j in values:
            prop_list = []
            # prop1
            if values[i,j] & 1:
                prop_list.append(('odd', True))

            # prop2
            for x,y in ((0,1),(0,-1), (1,0), (-1,0)):
                if (i+x,j+y) in values and values[i+x,j+y] < values[i,j]:
                    break
            else:
                prop_list.append(('min', True))

            # prop3
            if (i+1,j+1) in values and values[i+1,j+1] > values[i,j]:
                prop_list.append(('diag_inc', True))

            # prop4
            if 0 <= values[i,j] < 4:
                prop_list.append(('type', values[i,j]))

            props[i,j] = prop_list

        return props
    
    @classmethod
    def from_cells(self, cells):
        return ObjectWithProperties({ (i,j):cells.value[i,j] for i,j in cells.value })

    @staticmethod
    def addable_cells(obj):
        if obj.value:
            w = max(i for i,j in obj.value)+1
            h = max(j for i,j in obj.value)+1
            return list(set((i,j) for i in range(w+1) for j in range(h+1)) - set(obj.value))
        return [(0,0)]
    
    @staticmethod
    def removable_cells(obj):
        return list(obj.value)
    
    def add_cell(self, obj, pos, value, dirty={}):
        print("adapt_add_cell", pos, value, dirty)
        obj = ObjectWithProperties(obj.value)
        obj.value[pos] = value
        return obj
    
    #@staticmethod
    def remove_cell(obj, pos, dirty={}):
        obj = ObjectWithProperties(obj.value)
        del obj.value[pos]
        return obj

In [ ]:
class StylePlaceHolder(HTML):
    def __init__(self):
        super(StylePlaceHolder, self).__init__()
        self.classes = {}

    def set_class(self, classname, attr, value):
        if value is None:
            return remove_class_attribute(classname, attr)
        self.classes.setdefault(classname,{})[attr] = value
        self.write()

    def remove_class_attribute(self, classname, attr):
        if classname in self.classes:
            if attr in self.classes[classname]:
                del self.classes[classname][attr]
            if not self.classes[classname]:
                del self.classes[classname]
        self.write()

    def remove_class(self, classname):
        if classname in self.classes:
            del self.classes[classname]
        self.write()

    def write(self):
        self.value = (
            '<style>%s</style>'
            % '\n'.join( '.{0}, button.{0}, .widget-text.{0} input {{{1}}}'.format(
                        classname,
                        ';'.join('%s:%s'
                                 % c
                                 for c in self.classes[classname].items()
                                )
                       )
                     for classname in self.classes
                    )
        )

    def clear_class_widget(self, widget, ignore=()):
        for classname in self.classes:
            if classname not in ignore:
                widget.remove_class(classname)

In [ ]:
def get_model_id(w):
    r"""
    For some reason, our widgets seem to lose their model_id
    This *hack* recovers it
    """
    for u in w.widgets:
        if w.widgets[u] == w:
            return u

class StyledGridView(GridViewEditor, VBox, ValueWidget):
    def __init__(self, obj, adapter=None, display_convention='en', cell_layout=Layout(),
                 cell_widget_class=TextCell, addable_widget_class=AddableTextCell, **kwargs):
        GridViewEditor.__init__(self, obj, adapter)
        VBox.__init__(self, **kwargs)
        
        self._model_id = get_model_id(self)
        
        self.cell_layout = cell_layout
        self.empty_cell = EmptyCell(width=cell_layout.width, height=cell_layout.height)
        
        self.last_uid = 1
        self.cell_uid_to_cell_pos = {}
        self.cell_pos_to_cell_uid = {}
        self.cells_widgets = {}
        self.last_addable_position = []
        
        self.style_placeholder = StylePlaceHolder()
        self.style_class_names = []
        self.vbox = VBox()
        self.children = [self.style_placeholder, self.vbox]
        
        self.cell_widget_class = cell_widget_class
        self.addable_widget_class = addable_widget_class
        
        try:
            self.displaytype = cell_widget_class.displaytype
        except:
            self.displaytype = None # Stateless cells
        
        self.properties = []
        
        self.draw()
        self.donottrack = False
        
        for cls in ('addable_cell','removable_cell'):
            for attr,val in (('background-position', 'right top'),
                             ('background-size', '1em'), 
                             ('background-repeat', 'no-repeat')):
                self.style_placeholder.set_class(cls, attr, val)
        self.style_placeholder.set_class('addable_cell', 'background-image', "url('data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEYAAAA8BAAAAAA7DH7+AAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAAAnRSTlMAAHaTzTgAAAACYktHRAAPOjI+owAAAAlwSFlzAAAN1wAADdcBQiibeAAAAAd0SU1FB+MCBRIvKscf5nsAAAAtSURBVEjHY2AYBaSBmTNnThhVM6pmaKvhnIkVTBjqagZbOI+qGVVDHTXDAwAAnSews7vfUfkAAAAldEVYdGRhdGU6Y3JlYXRlADIwMTktMDItMDVUMTc6NDc6NDIrMDE6MDDD7zGLAAAAJXRFWHRkYXRlOm1vZGlmeQAyMDE5LTAyLTA1VDE3OjQ3OjQyKzAxOjAwsrKJNwAAAABJRU5ErkJggg==')")
        self.style_placeholder.set_class('removable_cell', 'background-image', "url('data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEYAAAA8BAAAAAA7DH7+AAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAAAnRSTlMAAHaTzTgAAAACYktHRAAPOjI+owAAAAlwSFlzAAAN1wAADdcBQiibeAAAAAd0SU1FB+MCBRIvL7d1EvQAAAAgSURBVEjHY2AYBaNgFIwwwDkTK5gw1NWMglEwCgYxAAAoCFJ7GFQEKQAAACV0RVh0ZGF0ZTpjcmVhdGUAMjAxOS0wMi0wNVQxNzo0Nzo0NyswMTowMJHXHiwAAAAldEVYdGRhdGU6bW9kaWZ5ADIwMTktMDItMDVUMTc6NDc6NDcrMDE6MDDgiqaQAAAAAElFTkSuQmCC')")
        
    def cast(self, value):
        try:
            return self.adapter.celltype(value)
        except:
            return self.adapter.cellzero

    def add_property(self, prop):
        if prop not in self.properties:
            self.properties.append(prop)
            self.apply_properties()

    def set_property_attr(self, prop, attr, value):
        self.add_property(prop)
        self.style_placeholder.set_class(prop.get_css_name(), attr, value)

    def apply_properties(self):
        props = self.adapter.get_properties(self.value)

        for pos in self.cells:
            widget = self.get_cell_widget(pos)
            self.style_placeholder.clear_class_widget(widget, ignore=('addable_cell','removable_cell'))
            for p in self.properties:
                if pos not in props:
                    continue
                for v in props[pos]:
                    if p.has_property(*v):
                        widget.add_class(p.get_css_name())
                        break

    def set_cell_layout(self, layout):
        self.cell_layout = layout
        self.empty_cell.layout.width = self.cell_layout.width
        self.empty_cell.layout.height = self.cell_layout.height

        for row in self.vbox.children:
            for cell in row.children:
                cell.layout.width = self.cell_layout.width
                cell.layout.height = self.cell_layout.height

    def _clear(self):
        self.vbox.children = ()
        self.reset_links()

    def _strip(self):
        """
        Remove empty line ending and ending empty lines.
        """
        for line in self.vbox.children:
            last = 0
            for i,elem in enumerate(line.children):
                if elem != self.empty_cell:
                    last = i
            if last+1 != len(line.children):
                line.children = line.children[:last+1]

        last = -1
        for i,elem in enumerate(self.vbox.children):
            if len(elem.children) != 1 or elem.children[0] != self.empty_cell:
                last = i
        if last+1 != len(self.vbox.children):
            self.vbox.children = self.vbox.children[:last+1]
            
    def _remove_trait_link(self, traitname):
        for l in list(self.links):
            if l.target == (self, traitname):
                l.unlink()
                self.links.remove(l)

    def _add_cell(self, pos, widget, traitname):
        new_uid = self.last_uid + 1
        self.cell_pos_to_cell_uid[pos] = new_uid
        self.cell_uid_to_cell_pos[new_uid] = pos
        self.cells_widgets[new_uid] = widget

        self.last_uid = new_uid

        self._remove_trait_link('cell_%d_%d' % pos)
        self._remove_trait_link('add_%d_%d' % pos)
        
        if widget and hasattr(widget, 'value'):
            link = dlink((widget, 'value'), (self, traitname), self.cast)
            self.links.append(link)

    def get_cell_widget(self, pos):
        if pos in self.cell_pos_to_cell_uid:
            return self.cells_widgets[self.cell_pos_to_cell_uid[pos]]

    def get_cell_widgets(self):
        yield from self.cells_widgets.values()
        
    def draw(self, **kwargs):
        removable_cells = self.removable_cells()
        addable_cells = self.addable_cells()
        
        for pos in list(self.cell_pos_to_cell_uid):
            if pos not in self.cells and pos not in addable_cells: 
                self.remove_cell_widget(pos)
        
        for pos in addable_cells:
            if pos not in self.last_addable_position:
                self.set_cell_widget(pos, self.addable_widget_class(), 'add_%d_%d' % pos)
        
        for pos in self.cells:
            widget = self.get_cell_widget(pos)
            new_value = self.adapter.cell_to_display(self.cells[pos], self.displaytype)
            if type(widget) != self.cell_widget_class or widget.value != new_value:
                widget = self.cell_widget_class(new_value)
                self.set_cell_widget(pos, widget, 'cell_%d_%d' % pos)
            elif widget.value != new_value:
                widget.value = new_value
            if pos in removable_cells:
                widget.add_class('removable_cell')
                if issubclass(self.cell_widget_class, ToggleButton):
                    widget.disabled = False
            else:
                widget.remove_class('removable_cell')    
                if issubclass(self.cell_widget_class, ToggleButton):
                    widget.disabled = self.cell_widget_class(True).disabled
                
        self.last_addable_position = addable_cells
        self.apply_properties()

    def set_cell_widget(self, pos, widget, traitname):
        """
        Define the widget on position `pos`
        """
        widget.layout.width = self.cell_layout.width
        widget.layout.height = self.cell_layout.height
        x,y = pos

        # remove old uid
        if pos in self.cell_pos_to_cell_uid:
            old_uid = self.cell_pos_to_cell_uid[pos]
            del self.cells_widgets[old_uid]
            del self.cell_uid_to_cell_pos[old_uid]

        self._add_cell(pos, widget, traitname)

        if x >= len(self.vbox.children):
            self.vbox.children += tuple(HBox([self.empty_cell]) for _ in range(x - len(self.vbox.children) + 1))
        line = self.vbox.children[x]

        if y >= len(line.children):
            line.children += (self.empty_cell,) * (y - len(line.children) + 1)

        line.children = line.children[:y] + (widget,) + line.children[y+1:]

    def remove_cell_widget(self, pos):
        """
        Delete the widget at position `pos` leaving a empty space
        """
        x,y = pos

        if x < len(self.vbox.children) and y < len(self.vbox.children[x].children):
            # remove cell
            if pos in self.cell_pos_to_cell_uid:
                old_uid = self.cell_pos_to_cell_uid[pos]
                del self.cells_widgets[old_uid]
                del self.cell_uid_to_cell_pos[old_uid]
                del self.cell_pos_to_cell_uid[pos]
                
                self._remove_trait_link('cell_%d_%d' % pos)
                self._remove_trait_link('add_%d_%d' % pos)
                
            widget = self.vbox.children[x].children[y]
            self.vbox.children[x].children = self.vbox.children[x].children[:y] + (self.empty_cell,) + self.vbox.children[x].children[y+1:]
            
            self._strip()
            
            return widget


In [ ]:
cell_layout = Layout(width='28px')
obj = ObjectWithProperties({(0,0):1})
g = StyledGridView(obj, ObjectGridViewAdapter(), cell_layout=cell_layout)
g

In [ ]:
g.value = ObjectWithProperties( {(0,0):2,(0,1):4} )

In [ ]:
PropOdd = Property('odd')
PropMin = Property('min')
PropInc = Property('diag_inc')
PropTypes = [ Property('type', cond=lambda x: x == i) for i in range(4) ]

In [ ]:
for p in PropTypes + [PropOdd, PropMin, PropInc]:
    g.add_property(p)
g.set_property_attr(PropOdd, 'background-color', 'green')
g.set_property_attr(PropMin, 'color', 'red')


In [ ]:
from sage.all import ParallelogramPolyomino, DyckWords
from sage_widget_adapters.combinat.parallelogram_polyomino_grid_view_adapter import ParallelogramPolyominoGridViewAdapter

In [ ]:
class ParallelogramPolyominoAdapter(ParallelogramPolyominoGridViewAdapter):
    @staticmethod
    def get_bounce_cells_from_pp(pp):
        ret = []
        x,y = 0,-1
        for i,v in enumerate(pp.bounce_path()):
            dx,dy = (1,0) if i & 1 else (0,1) 
            for j in range(v):
                x,y = x+dx, y+dy
                ret.append((i,(x,y)))
        return ret
    
    @staticmethod
    def get_properties(obj):
        props = {}
        
        # Values of the bounce path
        for i,pos in ParallelogramPolyominoAdapter.get_bounce_cells_from_pp(obj):
            props[pos] = [('bounce0', True)]

        return props

In [ ]:
pp = ParallelogramPolyomino.from_dyck_word(DyckWords(10).random_element())

In [ ]:
class DisabledButton(ToggleButton):
    def __init__(self, value, *args, **kwargs):
        super(DisabledButton, self).__init__(*args, **kwargs)
        self.disabled = True 
        self.value = value

class AddableButton(ToggleButton):
    def __init__(self, *args, **kwargs):
        super(AddableButton, self).__init__(*args, **kwargs)
        self.add_class('addable_cell')
    

In [ ]:
gp = StyledGridView(pp, ParallelogramPolyominoAdapter(), cell_layout=cell_layout, 
                    cell_widget_class=DisabledButton, 
                    addable_widget_class=AddableButton)

In [ ]:
gp

In [ ]:
PropBounce = Property('bounce0')
gp.add_property(PropBounce)
gp.set_property_attr(PropBounce, 'background-color', 'red')